导入包及模块

In [42]:
import torch
import numpy as np

生成数据集

In [43]:
num_inputs=2
num_examples=1000
true_w=[2,-2.3]
true_b=4.2

features=torch.randn(num_examples,num_inputs,dtype=torch.float32)
labels = true_w[0]*features[:,0]+true_w[1]*features[:,1]+true_b
labels += torch.tensor(np.random.normal(0,0.001),dtype=torch.float32)

读取数据

In [44]:
import torch.utils.data as data
batchSize=10
dataset=data.TensorDataset(features,labels)
#随机读取小批量
dataIter=data.DataLoader(dataset,batchSize,shuffle=True)

定义模型

In [45]:
import torch.nn as nn
class linearNet(nn.Module):
    def __init__(self,featureNum):
        super().__init__()
        self.linear=nn.Linear(featureNum,1)
    def forward(self,x):
        y=self.linear(x)
        return y

查看模型结构

In [46]:
net = linearNet(num_inputs)
print(net) 

linearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


其他定义模型结构的方式

In [47]:
#写法1
net=nn.Sequential(
    nn.Linear(num_inputs,1)
)
#写法2
net=nn.Sequential()
net.add_module('linear',nn.Linear(num_inputs,1))
#写法3
from collections import OrderedDict
net=nn.Sequential(
    OrderedDict(
        [
            ('linear',nn.Linear(num_inputs,1))
        ]
    )
)

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


查看模型参数

In [48]:
#net.parameters()查看模型所有的可学习参数
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.5777,  0.1922]], requires_grad=True)
Parameter containing:
tensor([0.2188], requires_grad=True)


初始化模型参数  
__init模块中提供了多种参数初始化方法___

In [49]:
from torch.nn import init

init.normal_(net[0].weight,mean=0,std=0.01)
init.constant_(net[0].bias,val=0)

Parameter containing:
tensor([0.], requires_grad=True)

定义损失函数  
__PyTorch在nn模块中提供了各种损失函数__

In [50]:
#均方误差
loss = nn.MSELoss()

定义优化算法  
__torch.optim模块提供了很多常用的优化算法比如SGD、Adam和RMSProp等__

In [51]:
# 小批量随机梯队下降优化算法
import torch.optim as optim
optimzer = optim.SGD(net.parameters(),lr=0.03)
print(optimzer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


还可以为子网络设置不同的学习率

In [52]:
optimizer =optim.SGD([
                # 如果对某个参数不指定学习率，就使用最外层的默认学习率
                {'params': net.subnet1.parameters()}, # lr=0.03
                {'params': net.subnet2.parameters(), 'lr': 0.01}
            ], lr=0.03)

ModuleAttributeError: 'Sequential' object has no attribute 'subnet1'

有时候我们不想让学习率固定成一个常数，那如何调整学习率呢？主要有两种做法。  
__修改optimizer.param_groups中对应的学习率__  
__新建优化器__

In [105]:
#调整学习率
for param in optimzer.param_groups:
    print(param['lr'])

0.03


训练模型

In [29]:
epochsNum=3
for epoch in range(epochsNum):
    for x,y in dataIter:
        output=net(x)
        l=loss(output,y.view(-1,1))
        l.backward()
        optimzer.step()
        optimzer.zero_grad()
    
    print(epoch,l.item())

0 7.643397199885715e-12
1 7.790390728346086e-12
2 7.754863591558081e-12


In [30]:
print(true_w,net[0].weight)

[2, -2.3] Parameter containing:
tensor([[ 2.0000, -2.3000]], requires_grad=True)
